In [1]:
import torch
import torch.nn as nn
from torchvision import datasets ,models,transforms
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.nn import Linear, ReLU, CrossEntropyLoss, Conv2d, MaxPool2d, Module
from torch.optim import Adam
import pandas as pd
import os
from os import listdir
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from dataloader_teeth_kfold import *
from torch.utils.data import DataLoader
from torchsummary import summary
import torch.optim as optim
from torch.nn import *
from EarlyStopping import EarlyStopping
from models import *
#from pytorchtools import EarlyStopping

In [2]:
%%time
batch_size = 16
train_dataset = GetDataset('','train')
val_dataset = GetDataset('','val')
test_dataset = GetDataset('','test',Istestori=True)
trainloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
valloader = DataLoader(val_dataset, batch_size=batch_size,shuffle=True)
testloader = DataLoader(test_dataset, batch_size=1,shuffle=False)
print('train :{}\t,test :{}\t,val :{}\t'.format(len(train_dataset),len(test_dataset),len(val_dataset)))

data length = 191040
> Found 30000 images...
data length = 191040
> Found 10000 images...
data length = 1912
> Found 1912 images...
train :30000	,test :1912	,val :10000	
Wall time: 19.6 s


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
model=DeepConvNet().to(device)
print(summary(model, (1, 200, 150)))
# model.eval()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 148]             320
            Conv2d-2         [-1, 32, 196, 146]           9,248
       BatchNorm2d-3         [-1, 32, 196, 146]              64
              ReLU-4         [-1, 32, 196, 146]               0
         MaxPool2d-5           [-1, 32, 98, 73]               0
            Conv2d-6           [-1, 64, 96, 71]          18,496
       BatchNorm2d-7           [-1, 64, 96, 71]             128
              ReLU-8           [-1, 64, 96, 71]               0
         MaxPool2d-9           [-1, 64, 48, 35]               0
           Conv2d-10          [-1, 128, 46, 33]          73,856
      BatchNorm2d-11          [-1, 128, 46, 33]             256
             ReLU-12          [-1, 128, 46, 33]               0
        MaxPool2d-13          [-1, 128, 23, 16]               0
           Conv2d-14          [-1, 128,

In [5]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, title, fold_num):
    
    confusion = confusion_matrix(y_true, y_pred)
    classes = [i for i in range(1,33)]
    confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis] #normalize
    fig, ax = plt.subplots(figsize=(15,15))
    im = ax.imshow(confusion, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)

    ax.set(xticks=np.arange(confusion.shape[1]),
           yticks=np.arange(confusion.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    
    fmt = '.2f'
    thresh = confusion.max() / 2.
    for i in range(confusion.shape[0]):
        for j in range(confusion.shape[1]):
            ax.text(j, i, format(confusion[i, j], fmt),
                    ha="center", va="center",
                    color="white" if confusion[i, j] > thresh else "black")
    fig.tight_layout()
    plt.savefig('plot/'+str(fold_num) + '_' + title + ".png")
    plt.clf()
    plt.cla()
    plt.close()
    
    return ax

In [6]:
def rotate(l, n):
    return l[-n:] + l[:-n]

def plot_acc_loss(train_acc,val_acc,train_loss,val_loss,fold_num):
    fig = plt.figure(figsize=(8,4))
    plt.plot(train_acc,label='Train_acc')
    plt.plot(val_acc,label='Val_acc')
    plt.grid(True)
    plt.title(str(fold_num))
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy(%)')
    plt.legend(loc='upper left',fontsize=8)
    plt.savefig('plot/'+str(fold_num)+'cc.png')
    
    fig = plt.figure(figsize=(8,4))
    plt.plot(train_loss,label='Train_loss')
    plt.plot(val_loss,label='Val_loss')
    plt.grid(True)
    plt.title(str(fold_num))
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='upper left',fontsize=8)
    plt.savefig('plot/'+str(fold_num)+'loss.png')

In [7]:
def create_nonflip(testloader, fold_num):
    count = 0
    df = pd.DataFrame(columns=['folder','image','label','predict','total_path'])
    for idx, data in tqdm(enumerate(testloader)):
        img, label, img_path = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64), data[2]
        if img_path[0][-5] == 'F':
            continue
        x, y = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64)
        output = model(x)
        max_value, predicted = torch.max(output.data, 1)

        if len(img_path[0].split('_'))==14:
            foloder = img_path[0].split('_')[-7]
        if len(img_path[0].split('_'))==15:
            foloder = img_path[0].split('_')[-8] + '_' + img_path[0].split('_')[-7]

        image = img_path[0].split('_')[-3]
        label = label.cpu().numpy()[0]+1
        predict = predicted.cpu().numpy()[0]+1
        df.loc[count] = [foloder, image, label, predict, img_path[0]]
        count += 1

    df = df.set_index(keys = ['folder','image'])
    df.sort_values(['folder', 'image','label'],inplace=True)

    for idx in df.index.unique():
        if df.loc[idx]['label'][0] > 16:
            df.loc[idx] = df.loc[idx].sort_values(['folder', 'image','label'],ascending=[True, True, False])
    df.to_csv('kfold/'+str(fold_num)+'_non_flip'+'.csv')
    
    len_dataframe = pd.DataFrame(columns=['folder','image','length'])
    for idx, folder_image in enumerate(df.index.unique()):
        len_dataframe.loc[idx] = [folder_image[0],folder_image[1],len(df.loc[folder_image])]
    len_dataframe = len_dataframe.set_index(keys = ['folder','image'])
    len_dataframe.to_csv('kfold/'+str(fold_num)+'len_dataframe.csv')
    

In [8]:
def revised_by_table(radiograph, uni):
    if (uni==4) or (uni==5):
        radiograph.iloc[0] = [4,13,True]
        radiograph.iloc[1] = [5,12,True]
    elif (uni==8) or (uni==9):
        radiograph.iloc[0] = [8,9,True]
        radiograph.iloc[1] = [9,8,True]
    elif (uni==12) or (uni==13):
        radiograph.iloc[0] = [12,5,True]
        radiograph.iloc[1] = [13,4,True]
    elif (uni==20) or (uni==21):
        radiograph.iloc[0] = [21,28,True]
        radiograph.iloc[1] = [20,29,True]
    elif (uni==28) or (uni==29):
        radiograph.iloc[0] = [29,20,True]
        radiograph.iloc[1] = [28,21,True]
    return radiograph

In [9]:
def count_point_table(radiograph):
    point_table = np.zeros(len(radiograph),dtype=int)
    #print('len(point_table) =',len(radiograph))
    
    upper = 0
    lower = 0
    for radio_idx, radio_row in radiograph.iterrows():#上下排預測
        if radio_row['p'] < 17:
            upper +=1
        else:
            lower += 1
        if radio_row['fp'] < 17:
            upper +=1
        else:
            lower += 1
#     if radiograph['p'].mean() < 17:
    if upper >= lower:
        v_value = 17
        for radio_idx, radio_row in radiograph.iterrows():#加起來17
            if radio_row['p']+radio_row['fp'] == 17:
                point_table[radio_idx] += 1

        for i in range(len(radiograph)-1): #正向分數
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['p'] - radiograph.iloc[i]['p'] == j - i:
                    point_table[i] += 1
                    point_table[j] += 1

        for i in range(len(radiograph)-1): #逆向分數
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['fp'] - radiograph.iloc[i]['fp'] == i - j:
                    point_table[i] += 1
                    point_table[j] += 1
    else:
        v_value = 49
        for radio_idx, radio_row in radiograph.iterrows():#加起來49
            if radio_row['p']+radio_row['fp'] == 49:
                point_table[radio_idx] += 1

        for i in range(len(radiograph)-1):
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['p'] - radiograph.iloc[i]['p'] == i - j:
                    point_table[i] += 1
                    point_table[j] += 1

        for i in range(len(radiograph)-1):
            for j in range(i+1,len(radiograph)):
                if radiograph.iloc[j]['fp'] - radiograph.iloc[i]['fp'] == j - i:
                    point_table[i] += 1
                    point_table[j] += 1

    #print(point_table)
    return (point_table, v_value)

def revised(radiograph, point_table, v_value):
    max_num = point_table.max()
    if len(point_table) == 1:
        return radiograph
    
    base = 0
    for i in range(len(point_table)):
        if point_table[i]==max_num:
            base = i
            break
    
    if v_value == 17:
        for radio_idx in range(len(point_table)):
#             gap = radio_idx - base
#             revise_value = radiograph.iloc[base]['p']+gap
#             radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False]
            if point_table[radio_idx]!=max_num:
                gap = radio_idx - base
                revise_value = radiograph.iloc[base]['p']+gap
                radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False]
            else:
                base = radio_idx
    else:
        for radio_idx in range(len(point_table)):
#             gap = radio_idx - base
#             revise_value = radiograph.iloc[base]['p']-gap
#             radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False] 
            if point_table[radio_idx]!=max_num:
                gap = radio_idx - base
                revise_value = radiograph.iloc[base]['p']-gap
                radiograph.iloc[radio_idx] = [revise_value,v_value-revise_value,False] 
            else:
                base = radio_idx 
            
    return radiograph

In [10]:
def algorithm(fold_num):
    new_size = (200,150)
    trans_method = [
        transforms.Resize(new_size),
        transforms.Grayscale(),
        transforms.ToTensor()
    ]
    trans_flipback = [
        transforms.Resize(new_size),
        transforms.RandomVerticalFlip(p=1),
        transforms.Grayscale(),
        transforms.ToTensor()
    ]
    trans = transforms.Compose(trans_method)
    trans_back = transforms.Compose(trans_flipback)
    
    len_dataframe = pd.read_csv('kfold/'+str(fold_num)+'len_dataframe.csv')
    len_dataframe = len_dataframe.set_index(keys = ['folder','image'])
    df = pd.read_csv('kfold/'+str(fold_num)+'_non_flip'+'.csv')
    df = df.set_index(keys = ['folder','image'])
    
    model = torch.load('./model/'+'fold'+str(fold_num)+'/best.pkl')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#=================================================================    
    count = 0
    cur_index = len_dataframe.iloc[count].values[0]

    out_df = pd.DataFrame(columns=['folder','image','label','flip_label','p','fp','check'])
    out_df_idx = 0

    radiograph = pd.DataFrame(columns=['p','fp','check'])
    radiograph_idx = 0
    ori_df = pd.DataFrame(columns=['folder','image','ori_p','ori_fp','prob_p','prob_fp'])

    total_point_table = []

    for idx, row in enumerate(df.iterrows()):
        if idx >= cur_index: # next radiograph

            radiograph_idx = 0
            radiograph = pd.DataFrame(columns=['p','fp','check'])

            count += 1
            cur_index += len_dataframe.iloc[count].values[0]
        img_path = row[1]['total_path']
        img = PIL.Image.open(img_path).convert('RGB')
        flip_img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)

        label = row[1]['label']
        if label < 17:
            flabel = 17 - label
            img = trans_back(img)
            flip_img = trans_back(flip_img)
        else:
            flabel = 49 - label
            img = trans(img)
            flip_img = trans(flip_img)

        img = img.to(device)
        img.unsqueeze_(0)
        flip_img = flip_img.to(device)
        flip_img.unsqueeze_(0)

        output = model(img)

        max_value, predicted = torch.max(output.data, 1)
        prob_p = F.softmax(output.data,dim=1).cpu().numpy().max()
        
        output = model(flip_img)
        max_value, flip_predicted = torch.max(output.data, 1)
        prob_fp = F.softmax(output.data,dim=1).cpu().numpy().max()
        
        p = predicted.cpu().numpy()[0]+1
        fp = flip_predicted.cpu().numpy()[0]+1

        ori_df.loc[idx] = [row[0][0],row[0][1],p,fp,prob_p,prob_fp]
        
        if (p+fp==17) | (p+fp==49):
            check = True
        else:
            check = False

        radiograph.loc[radiograph_idx] = [p, fp, check]
        radiograph_idx += 1
        if idx == cur_index-1: # do rule operation
            #print('size =',len_dataframe.iloc[count].values[0])

#             print(row[0])

            #全錯先不處理
            if len(radiograph[radiograph['check']==False]) == len(radiograph):
                for i in range(len(radiograph)):

                    label = df.loc[row[0],['label']].iloc[i][0]
                    if label < 17:
                        flabel = 17 - label
                    else:
                        flabel = 49 - label

                    out_df.loc[out_df_idx] = [row[0][0],row[0][1],label,flabel,radiograph.iloc[i]['p'],radiograph.iloc[i]['fp'],radiograph.iloc[i]['check']]
                    out_df_idx += 1

                    total_point_table+=[999]

                continue

            point_table, v_value = count_point_table(radiograph)

            radiograph = revised(radiograph, point_table, v_value)

            if (len(radiograph[radiograph['check']==True]) == len(radiograph)) and (len(radiograph)==2):
                if len(radiograph['p'].unique()) == 1:
                    uni = radiograph['p'].unique()[0]
                    radiograph = revised_by_table(radiograph,uni)

            for i in range(len(radiograph)):
                label = df.loc[row[0],['label']].iloc[i][0]
                if label < 17:
                    flabel = 17 - label
                else:
                    flabel = 49 - label
                out_df.loc[out_df_idx] = [row[0][0],row[0][1],label,flabel,radiograph.iloc[i]['p'],radiograph.iloc[i]['fp'],radiograph.iloc[i]['check']]
                out_df_idx += 1
            total_point_table += point_table.tolist()
            radiograph['ori_p'] = df.loc[row[0],['label']].values
            radiograph['ori_fp'] = v_value - df.loc[row[0],['label']].values
            
#             if row[0][1] == '15C4DA':
#                 print(pd.DataFrame(radiograph))
#             print()
    out_df['point'] = total_point_table
    out = pd.concat([out_df,ori_df[['ori_p','ori_fp','prob_p','prob_fp']]],axis=1)
    for radio_idx, radio_row in out_df.iterrows():
        if radio_row['check'] == True:
            out_df.loc[radio_idx,'check'] = 1
    out_df = out_df.reindex(columns=['folder','image','label','p','flip_label','fp','check','point'])
    out_df = out_df.set_index(keys = ['folder','image'])
    out_df.to_csv('kfold/'+str(fold_num)+'_revised_prediction.csv')
    ori_df = ori_df.set_index(keys = ['folder','image'])
    out = out.reindex(columns=['folder','image','label','p','flip_label','fp','check','ori_p','ori_fp','point','prob_p','prob_fp'])
    out = out.set_index(keys = ['folder','image'])
    out.to_csv('kfold/'+str(fold_num)+'_ori_pred.csv')
    acc_p = 0
    acc_fp = 0
    for idx, row in out.iterrows():
        if row['ori_p'] == row['label']:
            acc_p += 1
        if row['ori_fp'] == row['flip_label']:
            acc_fp += 1
    acc = acc_p + acc_fp
    print('=============original acc===============')
    print(acc_p/len(out_df), acc_fp/len(out_df),acc / (2*len(out_df)))
    acc_p = 0
    acc_fp = 0
    for idx, row in out.iterrows():
        if row['p'] == row['label']:
            acc_p += 1
        if row['fp'] == row['flip_label']:
            acc_fp += 1
    acc = acc_p + acc_fp
    print(acc_p/len(out_df), acc_fp/len(out_df),acc / (2*len(out_df)))
    
    x = np.concatenate((out_df['p'].values.astype(int),out_df['fp'].values.astype(int)))
    y = np.concatenate((out_df['label'].values.astype(int),out_df['flip_label'].values.astype(int)))
    plot_confusion_matrix(y, x,'revised_confusion',fold_num)
    
    

In [11]:
def evaluate(testloader, fold_num):
    model = torch.load('./model/'+'fold'+str(fold_num)+'/best.pkl')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.eval()

    total = 0
    correct = 0
    targets = []
    preds = []
    test_path = []
    for i, data in tqdm(enumerate(testloader)):
        x, y, img_path = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64), data[2]
        #test_path = test_path.append(data[2])

        output = model(x)
        max_value, predicted = torch.max(output.data, 1)

        total += y.size(0) #batch_size
        correct += (predicted == y).sum().item()

        targets.extend(y.view_as(predicted))
        preds.extend(predicted)

    print('Maxacc : %.4f'%(100*correct/total))
    targets_val = torch.stack(targets)
    preds_val = torch.stack(preds)
    plot_confusion_matrix(targets_val.cpu().numpy(), preds_val.cpu().numpy(),'original_confusion',fold_num)
    create_nonflip(testloader, fold_num)


In [12]:
import torchvision.models as models
def train(trainloader,valloader,epochs,learning_rate, patience,fold_num):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    #early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    # model
    model=DeepConvNet().to(device)
    
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-3)
    criterion = CrossEntropyLoss()

    train_acc=[]
    train_loss=[]
    
    val_acc=[]
    val_loss=[]
    
    total = 0
    correct = 0

    eval_total = 0
    eval_correct = 0
    
    max_vacc = 0
    
    print("Starting the training loop from epoch {}".format(0))
    
    for epoch in range(epochs):
        model.train()
        print('train')
        running_loss = 0
        total = 0
        correct = 0
        for data in tqdm(trainloader):
            
            # input
            x, y = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64)
#             print(x.shape)
            optimizer.zero_grad()
    
            output = model(x) # forward      
            
            #Calculate accuracy
            max_value, predicted = torch.max(output.data, 1)
            total += y.size(0) #batch_size
            correct += (predicted == y).sum().item()
            
            loss = criterion(output, y) #loss
            
            loss.backward() # backward
            optimizer.step()
            
            running_loss += loss.item()
        
        tloss=running_loss/len(trainloader)
        train_loss.append(tloss)
        
        tacc = correct/total
        train_acc.append(tacc)
        
        #==========eval===========
        model.eval()
#        print('eval')
        eval_total = 0
        eval_correct = 0
        val_running_loss = 0
        for data in tqdm(valloader):
            # input
            x, y = data[0].to(device, dtype=torch.float32), data[1].to(device, dtype=torch.int64)
#             optimizer.zero_grad()
            output = model(x) # forward
            #Calculate accuracy
            max_value, predicted = torch.max(output.data, 1)
            eval_total += y.size(0) #batch_size
            eval_correct += (predicted == y).sum().item()
            
            loss = criterion(output, y) #loss
            
            val_running_loss += loss.item()
        
        vloss=val_running_loss/len(valloader)
        val_loss.append(vloss)
        
        eval_acc = eval_correct/eval_total
        val_acc.append(eval_acc)
        
        if max_vacc < eval_acc:
            max_vacc = eval_acc
            torch.save(model, 'model/'+'fold'+str(fold_num)+'/'+'best'+'.pkl')
        
        print('Epoch[%d] Loss: %.4f'% (epoch+1, tloss))
        print('Epoch[%d] Acc: %.4f'% (epoch+1, tacc))
        print('Epoch[%d] eval_Acc: %.4f'% (epoch+1, eval_acc))        
        
        if int(eval_acc*100) >= 75:
            torch.save(model, 'model/'+'fold'+str(fold_num)+'/'+str(int(eval_acc*100))+'.pkl')
        
        
    print('max_eval_acc =,',max_vacc)
    return train_acc,train_loss,val_acc,val_loss

In [13]:
%%time
fold_nums = 5
ori_folds = [0,1,2,3,4]
choice_fold = [0,1,2,3,4]
# folds = rotate(folds,2)
batch_size = 16
n_epochs = 50
patience = 20
for fold_num in choice_fold:
    
    print('====='+str(fold_num)+ '===== ')
    folds = rotate(ori_folds,fold_num)
#     train_dataset = GetDataset('','train',folds=folds)
#     val_dataset = GetDataset('','val',folds=folds)
#     test_dataset = GetDataset('','test',Istestori=True,folds=folds)
#     trainloader = DataLoader(train_dataset, batch_size=batch_size,shuffle=True)
#     valloader = DataLoader(val_dataset, batch_size=batch_size,shuffle=True)
#     testloader = DataLoader(test_dataset, batch_size=1,shuffle=False)
    
#     train_acc,train_loss,val_acc,val_loss = train(trainloader,valloader,n_epochs,0.001, patience, fold_num)
#     plot_acc_loss(train_acc,val_acc,train_loss,val_loss,fold_num)
#     evaluate(testloader, fold_num)
    algorithm(fold_num)
    
#     folds = rotate(folds,1)

=====0===== 
=============original acc===============
0.805439330543933 0.8127615062761506 0.8091004184100419
0.8891213389121339 0.8891213389121339 0.8891213389121339
=====1===== 
=============original acc===============
0.7979057591623037 0.8052356020942408 0.8015706806282723
0.8827225130890053 0.8785340314136125 0.8806282722513089
=====2===== 
=============original acc===============
0.8020942408376963 0.7979057591623037 0.8
0.8774869109947644 0.8806282722513089 0.8790575916230366
=====3===== 
=============original acc===============
0.7895287958115184 0.7884816753926701 0.7890052356020942
0.8879581151832461 0.8848167539267016 0.8863874345549738
=====4===== 
=============original acc===============
0.806282722513089 0.8031413612565445 0.8047120418848167
0.8785340314136125 0.8827225130890053 0.8806282722513089
Wall time: 1min 51s


In [14]:
F.softmax(x.data,dim=1).cpu().numpy().max()

NameError: name 'x' is not defined

In [ ]:
pd.DataFrame(x.cpu().numpy()).T.sum()